In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Predictive_Maintenance_JetEngine'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    x_test_data_path: Path
    truth_value_path: Path
    model_path: Path
    metric_name: str

In [3]:
from PredictiveMaintenance.constants import CONFIG_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from PredictiveMaintenance.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_YAML_FILE_PATH,
        params_filepath = PARAMS_YAML_FILE_PATH,
        schema_filepath = SCHEMA_YAML_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostRegressor

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            x_test_data_path=config.x_test_data_path,
            truth_value_path=config.truth_value_path,
            model_path=config.model_path,
            metric_name=config.metric_name
        )

        return model_evaluation_config

In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
from PredictiveMaintenance import logger

In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_matrix(self, y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
        mse = mean_squared_error(y_pred=y_pred, y_true=y_true)
        r2 = r2_score(y_pred=y_pred, y_true=y_true)
        return rmse, mse, r2
    
    def save_matrix(self):
        X_test = pd.read_csv(self.config.x_test_data_path)

        truth_value_path = self.config.truth_value_path

        dfs = {}
        for filename in os.listdir(truth_value_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(truth_value_path, filename)

                df_name = os.path.splitext(filename)[0]

                df = pd.read_csv(file_path, sep="\s+", header=None, names=["RUL"])

                dfs[df_name] = df

                logger.info(f"DataFrame '{df_name}' has been created with shape: {df.shape}")
                logger.info(df.head())
        
        RUL_FD001 = dfs["RUL_FD001"]

        y_test = RUL_FD001.iloc[:,-1]

        logger.info(f"y_test: {y_test.shape}")

        model = joblib.load(self.config.model_path)
        y_pred = model.predict(X_test)

        (rmse, mse, r2) = self.eval_matrix(y_true=y_test, y_pred=y_pred)

        scores = {"RMSE": rmse, "MSE": mse, "r2_score": r2}
        save_json(path=Path(os.path.join(self.config.root_dir, self.config.metric_name)), data=scores)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    obj = ModelEvaluation(model_evaluation_config)
    obj.save_matrix()
except Exception as e:
    raise e

[2024-03-06 07:32:43,076: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-06 07:32:43,078: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-06 07:32:43,083: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-06 07:32:43,084: INFO: common: created directory at: artifacts]
[2024-03-06 07:32:43,085: INFO: common: created directory at: artifacts/model_evaluation]
[2024-03-06 07:32:43,089: INFO: 3398171951: DataFrame 'RUL_FD001' has been created with shape: (100, 1)]
[2024-03-06 07:32:43,090: INFO: 3398171951:    RUL
0  112
1   98
2   69
3   82
4   91]
[2024-03-06 07:32:43,092: INFO: 3398171951: DataFrame 'RUL_FD003' has been created with shape: (100, 1)]
[2024-03-06 07:32:43,093: INFO: 3398171951:    RUL
0   44
1   51
2   27
3  120
4  101]
[2024-03-06 07:32:43,094: INFO: 3398171951: y_test_FD001: (100,), y_test_FD003: (100,)]
[2024-03-06 07:32:43,095: INFO: 3398171951: Concatenating test data.]
[2024-03-06 07:32:43,111: INFO: comm